# fundus data

In [ ]:
import torch
def test(clf):
    print('\n Test')
    clf.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (images,labels) in enumerate(testloader):
            images,labels = images.cuda(),labels.cuda()
            outputs, _, _ = clf(images)
            #print(outputs)
            predicted = torch.argmax(outputs,dim=1)
            #print(labels)
            #print(predicted)
            total += labels.shape[0]
            correct += predicted.eq(labels).sum()
            print(batch_idx, len(testloader), ' Acc: %.3f%% (%d/%d)'
                % (100.*float(correct)/total, correct, total))

    acc = 100.*float(correct)/total
    return acc

In [1]:
import libadver.attack as attack
import net
import torch.backends.cudnn as cudnn
from torchvision import  transforms
import torchvision
import torch

pretrained_clf = net.AttnVGG(num_classes=2, attention=True, normalize_attn=True, dropout = 0.8)
pretrained_clf = torch.nn.DataParallel(pretrained_clf)
cudnn.benchmark = True

model_file = "messidor_attVGG_latest.pkl"
pretrained_clf.load_state_dict(torch.load(model_file))
pretrained_clf = pretrained_clf.cuda()
pretrained_clf.eval()

print("=====> Load Data")
testRootDir = "/store/dataset/messidor/binary_classification/test"
trainRootDir = "/store/dataset/messidor/binary_classification/train"
testBatchSize = 16
mean = (0.4914, 0.4822, 0.4465)
std = (0.2023, 0.1994, 0.2010)
test_transforms = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean,std)
])
testset = torchvision.datasets.ImageFolder(testRootDir,transform=test_transforms)
testloader = torch.utils.data.DataLoader(testset,batch_size=testBatchSize,shuffle=True,drop_last=False,num_workers=4)

trainset = torchvision.datasets.ImageFolder(trainRootDir,transform=test_transforms)
trainloader = torch.utils.data.DataLoader(trainset,batch_size=testBatchSize,shuffle=True,drop_last=False,num_workers=4)


#test(pretrained_clf)


/home/lrh/program/git/pytorch-example/Adversarial_Perturbations/adversarial/libadver/libadver/utils.py:46: UserWarning: Code Running Environment is not in Terminal, we can not invoke progress_bar!!
  warnings.warn("Code Running Environment is not in Terminal, we can not invoke progress_bar!!")


=====> Load Data


In [ ]:
import numpy as np
import torch
pgd_params = {
            'ord': np.inf,
            'y': None,
            'eps': 16.0 / 255,
            'eps_iter': 2.55 / 255,
            'nb_iter': 40,
            'rand_init': True,
            'rand_minmax': 16.0 / 255,
            'clip_min': 0.,
            'clip_max': 1.,
            'sanity_checks': True
        }


import libadver.attack as attack

PGDAttack = attack.ProjectGradientDescent(model = pretrained_clf)

correct = 0
total = 0
for image,label in trainloader:
    image, label = image.cuda(), label.cuda()
    ## non targeted
    pgd_params['y'] = label
    pgd_params['clip_min'] = torch.min(image) 
    pgd_params['clip_max'] = torch.max(image)
    
    adv_x = PGDAttack.generate(image, **pgd_params)
    #adv_x = image
    outputs, _, _ = pretrained_clf(adv_x)
    pred_adv = torch.argmax(outputs, dim = 1)
    
    for c in range(3):
        adv_x.data[:,c,:,:] = (adv_x.data[:,c,:,:] * std[c]) + mean[c]
        image.data[:,c,:,:] = (image.data[:,c,:,:] * std[c]) + mean[c]
    torchvision.utils.save_image(adv_x, "adv.png", nrow = 4)
    torchvision.utils.save_image(image, "image.png", nrow = 4)

    total = total + image.size(0)
    correct = correct + label.eq(pred_adv).sum()
    print("ACC: %.4f (%d, %d)" %(float(correct) / total, correct, total))

In [ ]:
import libadver.attack as attack

# Dermoscopy